In [1]:
import tensorflow as tf
import numpy as np

/home/ai/.local/share/virtualenvs/linghui-mEaGMfZL/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
# Explicitly create a Graph object
graph = tf.Graph()

with graph.as_default():
    with tf.name_scope("variables"):
        # Variable to keep track of how many times the graph has been run
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step")
        # Variable that keeps track of the sum of all output values over time:
        total_output = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="total_output")
    # Primary transformation Operations
    with tf.name_scope("transformation"):
        # Separate input layer
        with tf.name_scope("input"):
            # Create input placeholder- takes in a Vector 
            a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
        # Separate middle layer
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a, name="product_b")
            c = tf.reduce_sum(a, name="sum_c")
        # Separate output layer
        with tf.name_scope("output"):
            output = tf.add(b, c, name="output")
        
    with tf.name_scope("update"):
        # Increments the total_output Variable by the latest output
        update_total = total_output.assign_add(output)
        # Increments the above `global_step` Variable, should be run whenever the graph is run
        increment_step = global_step.assign_add(1)
    
    # Summary Operations
    with tf.name_scope("summaries"):
        # Compute the average output value over time
        # Use the output from update_total and increment_step to make sure that the updates happen
        # before we compute avg
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average") 
        # Creates summaries for output node
        tf.summary.scalar('output_summary', output)
        tf.summary.scalar('total_summary', update_total)
        tf.summary.scalar('average_summary', avg)
    
    # Global Variables and Operations
    with tf.name_scope("global_ops"):
        # Initialization Op
        init = tf.initialize_all_variables()    
        # Merge all summaries into one Operation
        merged_summaries = tf.summary.merge_all()

# Start a Session, using the explicitly created Graph
sess = tf.Session(graph=graph)

# Open a FileWriter to save summaries
writer = tf.summary.FileWriter('./improved_graph', graph)

# Initialize Variables
sess.run(init)

In [7]:
def run_graph(input_tensor):
    """
    Helper function; runs the graph with given input tensor and saves summaries
    """
    feed_dict = {a: input_tensor}
    out, step, summary = sess.run([output, increment_step, merged_summaries], feed_dict=feed_dict)
    # global step allows TensorBoard to graph data over time
    writer.add_summary(summary, global_step=step)

In [8]:
# Run the graph with various inputs
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

In [9]:
# Write the summaries to disk
writer.flush()

In [10]:
# Close the SummaryWriter
writer.close()

In [11]:
# Close the session
sess.close()

To start TensorBoard after running this code, run the following command:

```
$ tensorboard --logdir='./improved_graph'
```